In [76]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Make clicks stimulus

In [35]:
def rate_from_gamma(gamma, total_rate=40):
    left = total_rate / (np.exp( gamma) + 1);
    right = total_rate - left;
    return left, right

In [82]:
def plot_clicktrain(bups):
    left_bups = bups['left']
    right_bups = bups['right']
    left_rate = bups['left_rate']
    right_rate = bups['right_rate']
    duration = bups['duration']
    fig, ax = plt.subplots( figsize=(4,2.5))
    ax1 = ax
    ax1.eventplot(left_bups,lineoffsets=-.5,color="blue", alpha=.5)
    ax1.eventplot(right_bups,lineoffsets=.5,color="red", alpha=.5)
    ax1.plot(left_bups,-np.ones_like(left_bups), "o", color="blue", alpha=.5, label=f"r_L={left_rate:.2f} Hz")
    ax1.plot(right_bups,np.ones_like(right_bups), "o", color="red", alpha=.5, label=f"r_R={right_rate:.2f} Hz")
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Click sign")
    ax1.set_title("Clicks")
    ax1.legend(loc="upper center", ncol=2, bbox_to_anchor=(.5 ,1.5))
    ax1.set_xlim([0, duration])
    sns.despine()
    plt.tight_layout()
    plt.show()

def make_clicktrain(total_rate=40, gamma=1.5, duration=.5, dt=.001, stereo_click=True, rng=1):
    """Generate Poisson Click train"""

    tvec = np.arange(0, duration, dt)
    left_rate, right_rate = rate_from_gamma(gamma, total_rate)
    left_rate = total_rate - right_rate
    
    np.random.seed(rng)
    right_ind = np.random.random_sample(np.shape(tvec)) < (right_rate * dt)
    left_ind = np.random.random_sample(np.shape(tvec)) < (left_rate * dt)    
    
    if stereo_click:
        first_ind = np.argwhere(right_ind+left_ind>0)[0]
        right_ind[first_ind] = 1
        left_ind[first_ind] = 1
    
    left_bups = tvec[left_ind]
    right_bups = tvec[right_ind]
    
    bups = {'left':left_bups, 'right':right_bups, 'tvec':tvec, 
        'right_ind':right_ind, 'left_ind':left_ind, 'duration':duration,
           'left_rate':left_rate, 'right_rate':right_rate}
    
    plot_clicktrain(bups)
    
    return bups

In [83]:
# adapt the clicks
def make_adapted_clicks(bups, phi=.1, tau_phi=.2, cross_stream=True):
    if not cross_stream:
        raise notImplementedError
    if phi > 1:
        raise notImplementedError

    # concatenate left and right bups to get interclick intervals
    left_bups = bups['left']
    right_bups = bups['right']
    bups_cat = np.hstack([left_bups, right_bups])
    sign_cat = np.hstack([-np.ones_like(left_bups), np.ones_like(right_bups)])
    sort_order = np.argsort(bups_cat)
    bups_cat = bups_cat[sort_order]
    sign_cat = sign_cat[sort_order]
    ici = np.diff(bups_cat)
    
    C  = np.ones_like(bups_cat)
    cross_side_suppression = 0
    for ii in np.arange(1,len(C)):
        if ici[ii-1] <= cross_side_suppression and phi != 1:
            C[ii-1] = 0
            C[ii] = 0
            continue
        if abs(phi-1) > 1e-5:
            style = 'bing'
            if style=='bing':
                last = tau_phi  * np.log(abs(1 - C[ii-1] * phi))
                C[ii] = 1 - np.exp((-ici[ii-1] + last) / tau_phi)
            if style=='brian':
                arg = (1/tau_phi) * (-ici[ii-1] + sp.special.xlogy(tau_phi, abs(1.-C[ii-1]*phi)))
                if C[ii]*phi <=1:
                    C[ii] = 1. - np.exp(arg) 
                else:
                    C[ii] = 1. + np.exp(arg) 
    left_adapted = C[sign_cat==-1]
    right_adapted = C[sign_cat==1]
    bups['left_adapted'] = left_adapted
    bups['right_adapted'] = right_adapted
    
    # compute the full adaptation process
    tvec = bups['tvec']
    dt = np.mean(np.diff(tvec))
    Cfull = np.ones_like(tvec)

    for (ii, tt) in enumerate(tvec[:-1]):
        thislb = bups['left_ind'][ii] * 1.
        thisrb = bups['right_ind'][ii] * 1.
        if thislb + thisrb == 2. and phi != 1:
            Cfull[ii] = 0
        Cdot =  (1-Cfull[ii]) / tau_phi * dt + (phi - 1) * Cfull[ii] * (thislb + thisrb)
        Cfull[ii+1] = Cfull[ii] + Cdot

    fig, ax = plt.subplots(2, 1, figsize=(4,3), sharex=True)
    ax0 = ax[0]
    ax1 = ax[1]
    ax0.plot(tvec, Cfull, color = "gray")
    ax0.plot(left_bups, left_adapted, "o", color="blue", alpha=.5)
    ax0.plot(right_bups, right_adapted, "o", color="red", alpha=.5)
    ax0.set_ylabel("Adaptation value, C")
    
    ax1.plot(np.vstack([left_bups, left_bups]), 
             np.vstack([np.zeros_like(left_bups), -left_adapted]), color="blue", alpha=.5)
    ax1.plot(np.vstack([right_bups, right_bups]), 
             np.vstack([np.zeros_like(right_bups), right_adapted]), color="red", alpha=.5)
    
    ax1.plot(left_bups,-left_adapted, "o", color="blue", alpha=.5)
    ax1.plot(right_bups,right_adapted, "o", color="red", alpha=.5)
    ax1.set_xlabel("Time (s)")
    ax1.set_title("Adapted clicks")

    sns.despine()
    plt.tight_layout()

    plt.show()
    
    return None

In [84]:
def integrate_adapted_clicks(bups, lam=0, s2s=0.001, s2a=.001, s2i=.001, bias=0, B=5., nagents=5):
    tvec = bups['tvec']
    fig, ax = plt.subplots(figsize=(4,2))
    dt = np.mean(np.diff(tvec))
    dur = bups['duration']
    
    left_adapted = bups['left_adapted'].copy()
    right_adapted = bups['right_adapted'].copy()
    left_ts = bups['left']
    right_ts = bups['right']
    left_adapted *= np.exp(lam * (dur - left_ts))
    right_adapted *= np.exp(lam * (dur - right_ts))
    
    ax.set_xlim([0, dur])
    ax.axhline(bias,color='black',linestyle=':')
    ax.axhline(B,color='black',linestyle='-',lw=1)
    ax.axhline(-B,color='black',linestyle='-',lw=1)
    
    alims = [-1, 1]
    for agenti in np.arange(nagents):
        left_vals = np.zeros_like(tvec) 
        right_vals = np.zeros_like(tvec)
        
        left_vals[bups['left_ind']] =  left_adapted 
        right_vals[bups['right_ind']] = right_adapted 
        difflr = -left_vals + right_vals
        sumlr = left_vals + right_vals
        
        init_noise = np.random.normal(loc=0, scale=np.sqrt(s2i))
        a = np.zeros_like(tvec) + init_noise
        for ii in np.arange(len(tvec)-1):
            last_a = a[ii]

            adot = (dt * lam * last_a + 
                    difflr[ii] + 
                    np.random.normal(scale=np.sqrt(sumlr[[ii]]*s2s)) + 
                    np.random.normal(scale=np.sqrt(s2a*dt)))
            a[ii+1] = last_a + adot
        
        crossing = np.argwhere(abs(a)>B)
        if len(crossing ) > 0:
            ii = crossing[0][0]
            a[ii:] = np.ones_like(a[ii:])*np.sign(a[ii])*B
            
        alims[0] = min(alims[0],np.min(a)*1.1)
        alims[1] = max(alims[1],np.max(a)*1.1)
        
        ax.plot(tvec, a, color="pink")
        
    ax.set_ylim(alims)
    
    
    
    ax.set_xlabel("Time")
    ax.set_ylabel("Accumulation value, a")
    sns.despine()
    plt.show()
    return a


# Generate click trains
The left and right Poisson Click trains, $\delta_{t,t_L}$ and $\delta_{t,t_R}$ are generated by sampling at two different Poisson rates $r_L$ and $r_R$. 

These generative click rates are determined by setting the overall total rate (default value is 40Hz) and the contrast parameter $\gamma = \log \frac{r_R}{r_L}$.


In [95]:
def clicks_eventhandler(change):
    bups_out.clear_output(wait=True)
    adapted_out.clear_output(wait=True)
    with bups_out:
        bups = make_clicktrain(total_rate=rate_slider.value, gamma=gamma_slider.value, 
                               duration=dur_slider.value, rng=seed_slider.value)
    with adapted_out:
        make_adapted_clicks(bups, phi=phi_slider.value, tau_phi=tau_phi_slider.value)   
        
        
bups_out = widgets.Output()
adapted_out = widgets.Output()

seed_slider = widgets.IntSlider(value=1, min=0, max=10, description="Random Seed")
rate_slider = widgets.IntSlider(value=40,min=5,max=50,step=1,description='Total Rate (Hz)')
gamma_slider = widgets.FloatSlider(value=1.5,min=-5,max=5,step=.25,description=r"Gamma, $\gamma = \log \frac{r_1}{r_2}$")
dur_slider = widgets.FloatSlider(value=1,min=.25,max=5,step=.1,description="duration, T(s)")

phi_slider = widgets.FloatSlider(value=.1,min=.001,max=1,step=.05,description=r"$\phi$")
tau_phi_slider = widgets.FloatSlider(value=.15,min=.01,max=1,step=.05,description=r"$\tau_{\phi}$")
        
seed_slider.observe(clicks_eventhandler, names='value')
rate_slider.observe(clicks_eventhandler, names='value')
gamma_slider.observe(clicks_eventhandler, names='value')
dur_slider.observe(clicks_eventhandler, names='value')

phi_slider.observe(clicks_eventhandler, names='value')
tau_phi_slider.observe(clicks_eventhandler, names='value')
    
bup_inputs = widgets.VBox([seed_slider, rate_slider, gamma_slider, dur_slider])
adaptation_inputs = widgets.VBox([phi_slider, tau_phi_slider])



# Generate click train

In [96]:
clicktrain_io = widgets.HBox([bup_inputs, bups_out])
display(clicktrain_io) 

# Apply adaptation
$\frac{dC}{dt} = \frac{1-C}{\tau_{\phi}} + (\phi - 1) C (\delta_{t,t_R} + \delta_{t,t_L})$

In [97]:
adaptation_io = widgets.HBox([adaptation_inputs, adapted_out])
display(adaptation_io)

# Integrate the adapted clicks
Now, we need to integrate the adapted clicks. We'll also apply per-click noise (aka sensory noise) with standard deviation $\sigma_s$, memory noise with s.d.=$\sigma_a$, initial noise with s.d.=$\sigma_i$, and an integration time constant $\lambda$. This gives us our accumulation value
\begin{align}
da &= (\eta C \delta_R - \eta C \delta_L)dt + \lambda a dt + \sigma_adW , & a_0 = \mathcal{N}(0, \sigma_i^2)
\end{align}

In [ ]:
a = interactive(integrate_adapted_clicks, #{'manual': True},
                bups=fixed(bups), lam =(-5,5,.25), s2s=(0.001,10,.25), s2a=(.001, 5, .25), s2i=(.001,5,.1),
               bias=((-5,5,.1)), B=(0, 15, .5), nagents=(1,10))
display(a)

- [x] make bups sync
- [ ] put it online using sphinx or something?

In [ ]:
# TODO add lapse

In [ ]:
# TODO Make adaptation work for positive values of phi
# TODO Add tests for adaptation process

In [ ]:
# TODO fix the noise units

In [ ]:
# TODO add tests of adaptation
# put it on the internet 
# https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916


[link](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)

In [ ]:
# TODO add histogram and comparison to analytical model
# TODO save out a test dataset

In [ ]:
# TODO make bias a separate control